In [1]:
import torch


In [3]:
def load_model(checkpoint_path='checkpoint.pth'):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)

    # Extract the input size and other hyperparameters
    input_size = checkpoint['input_size']
    learning_rate = checkpoint['learning_rate']
    
    # Recreate the model
    model = Classifier(input_size=input_size)
    
    # Load the model state
    model.load_state_dict(checkpoint['state_dict'])
    
    # Recreate the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return model, optimizer 

In [4]:
import torch
from final_model import Classifier # # import the text file in which info is saved 

# Define the model and load the checkpoint
model, _ = load_model(checkpoint_path='checkpoint.pth')
model.eval()  # Set the model to evaluation mode

# Define a sample input tensor for tracing
sample_input = torch.tensor([[0.0606, 0.5000, 0.3333, 0.4828, 0.4000, 0.4000, 0.4000, 
                              0.4000, 0.4000, 0.4000, 0.1651, 0.0869, 0.0980, 0.1825, 
                              0.1054, 0.2807, 0.0016, 0.0000, 0.0033, 0.0027, 0.0031, 
                              0.0021]]).float()

# Create a TorchScript model using tracing
traced_model = torch.jit.trace(model, sample_input)
# standalone and do not require Python to run.
# can be loaded and run on different platforms, including mobile devices


# Save the traced model
traced_model.save('traced_model.pt')


In [5]:
# Load the traced model
traced_model = torch.jit.load('traced_model.pt')

# Define the input tensor
input_tensor = torch.tensor([[0.0606, 0.5000, 0.3333, 0.4828, 0.4000, 0.4000, 0.4000, 
                              0.4000, 0.4000, 0.4000, 0.1651, 0.0869, 0.0980, 0.1825, 
                              0.1054, 0.2807, 0.0016, 0.0000, 0.0033, 0.0027, 0.0031, 
                              0.0021]]).float()

# Perform prediction with the traced model
with torch.no_grad():  # Disable gradient calculation
    output = traced_model(input_tensor)
    _, predicted_class = torch.max(output, 1)

print(f"Predicted class: {predicted_class.item()}")


Predicted class: 1
